# Getting Live Data For The Current Season

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# API URL
# general
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
# per fixture
url_fix = 'https://fantasy.premierleague.com/api/fixtures/'
# per player
url_player = 'https://fantasy.premierleague.com/api/element-summary/{player-id}/'

In [4]:
# Use the requests package to make a GET request from the API endpoint
r = requests.get(url)

# transform that request into a json object
json = r.json()

# look at the json keys
json.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [5]:
elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

In [6]:
slim_elements_df = elements_df[['first_name','second_name','team','element_type','selected_by_percent','now_cost','transfers_in','value_season','total_points']]

In [7]:
# Map data to position
slim_elements_df['position'] = slim_elements_df.element_type.map(elements_types_df.set_index('id').singular_name)

# Map data to team name
slim_elements_df['team'] = slim_elements_df.team.map(teams_df.set_index('id').name)

# Add a column as player full_name
slim_elements_df['full_name'] = slim_elements_df.apply(lambda row: row['first_name'] + ' ' + row['second_name'], axis=1)

# Remove diacritics in players' name 
from unidecode import unidecode
slim_elements_df['full_name'] = slim_elements_df['full_name'].apply(unidecode)

In [8]:
slim_elements_df.head()

,first_name,second_name,team,element_type,selected_by_percent,now_cost,transfers_in,value_season,total_points,position,full_name
0,Folarin,Balogun,Arsenal,4,1.3,45,0,0.0,0,Forward,Folarin Balogun
1,Cédric,Alves Soares,Arsenal,2,0.4,40,0,2.5,10,Defender,Cedric Alves Soares
2,Mohamed,Elneny,Arsenal,3,0.2,45,0,1.3,6,Midfielder,Mohamed Elneny
3,Fábio,Ferreira Vieira,Arsenal,3,0.1,55,0,7.3,40,Midfielder,Fabio Ferreira Vieira
4,Gabriel,dos Santos Magalhães,Arsenal,2,26.3,50,0,29.2,146,Defender,Gabriel dos Santos Magalhaes


# Load in past seasons file

In [9]:
# Load in past seasons file
df = pd.read_excel('updated_file.xlsx')

In [48]:
df[df['preferred_name'] == 'Luke Shaw']

,index,full_name,preferred_name,ID,opponent_team,goals_scored,assists,clean_sheets,value,total_points,...,threat,ict_index,value.1,transfers_balance,selected,transfers_in,transfers_out,season_num,weights,weighted_points
103531,103532,Luke Paul Hoare Shaw,Luke Shaw,1230,Brighton,0,0,0,50,0,...,19,6.9,50,0,84371,0,0,season2223,1.00,0.00
103532,103533,Luke Paul Hoare Shaw,Luke Shaw,1230,Brentford,0,0,0,50,-1,...,0,2.1,50,-10978,90091,5529,16507,season2223,1.00,-1.00
103533,103534,Luke Paul Hoare Shaw,Luke Shaw,1230,Liverpool,0,0,0,49,0,...,0,0.0,49,-24730,68446,2162,26892,season2223,1.00,0.00
103534,103535,Luke Paul Hoare Shaw,Luke Shaw,1230,Southampton,0,0,0,48,0,...,0,0.0,48,-11746,60003,1558,13304,season2223,1.00,0.00
103535,103536,Luke Paul Hoare Shaw,Luke Shaw,1230,Leicester,0,0,0,48,0,...,0,0.0,48,-9027,52562,699,9726,season2223,1.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103792,103793,Luke Paul Hoare Shaw,Luke Shaw,1230,Everton,0,0,0,50,0,...,0,0.0,50,-48346,354892,744,49090,season1819,0.42,0.00
103793,103794,Luke Paul Hoare Shaw,Luke Shaw,1230,Man City,0,0,0,50,0,...,2,3.0,50,-48346,354892,744,49090,season1819,0.42,0.00
103794,103795,Luke Paul Hoare Shaw,Luke Shaw,1230,Chelsea,0,1,0,50,8,...,6,6.7,50,-2115,355402,5099,7214,season1819,0.42,3.36
103795,103796,Luke Paul Hoare Shaw,Luke Shaw,1230,Huddersfield,0,0,0,50,2,...,6,3.0,50,97811,456671,100609,2798,season1819,0.42,0.84


In [10]:
season_list = np.unique(df['season_num'])
weights = [0.14, 0.28, 0.42, 0.56, 0.70, 0.84, 1]

# Create a DataFrame
data_temp = {'season_num': season_list, 'weights': weights}
weight_df = pd.DataFrame(data_temp)
weight_df

,season_num,weights
0,season1617,0.14
1,season1718,0.28
2,season1819,0.42
3,season1920,0.56
4,season2021,0.70
5,season2122,0.84
6,season2223,1.00


In [11]:
# Add a column of weights values to the dataframe
merged_df = pd.merge(df, weight_df, left_on='season_num', right_on='season_num', how='left')

In [12]:
# Calculate weighted_points 
merged_df['weighted_points'] = merged_df['weights'] * merged_df['total_points']
merged_df.head()

,index,full_name,preferred_name,ID,opponent_team,goals_scored,assists,clean_sheets,value,total_points,...,threat,ict_index,value.1,transfers_balance,selected,transfers_in,transfers_out,season_num,weights,weighted_points
0,1,Aaron Anthony Connolly,Aaron Connolly,1,Burnley,0,0,0,55,0,...,0,0.0,55,0,15789,0,0,season2122,0.84,0.00
1,2,Aaron Anthony Connolly,Aaron Connolly,1,Watford,0,0,0,55,1,...,6,0.5,55,-1682,15599,1899,3581,season2122,0.84,0.84
2,3,Aaron Anthony Connolly,Aaron Connolly,1,Everton,0,0,0,55,0,...,0,0.0,55,-737,15457,2897,3634,season2122,0.84,0.00
3,4,Aaron Anthony Connolly,Aaron Connolly,1,Brentford,0,0,0,54,0,...,0,0.0,54,-3682,12065,1857,5539,season2122,0.84,0.00
4,5,Aaron Anthony Connolly,Aaron Connolly,1,Leicester,0,0,0,54,0,...,0,0.0,54,-2548,9659,301,2849,season2122,0.84,0.00


In [13]:
# Assign merged_df to df
df = merged_df

# Prepare functions to use 

In [14]:
import tkinter as tk
import pandas as pd


def trim_and_split(string):
    # Remove leading and trailing whitespace, split the string into words
    word_list = string.strip().split()
    return word_list


def string_contains_all_words(string, word_list):
    # Convert both the string and word list to lowercase
    string = string.lower()
    word_list = [word.lower() for word in word_list]

    # Check if all words in the word list are present in the string
    return all(word in string for word in word_list)


def search_dataframe(df, search_query):
    # Trim and split the search query
    query_words = trim_and_split(search_query)

    # Filter the DataFrame for exact matches
    exact_matches_mask = df["preferred_name"].apply(
        lambda x: x.lower() == search_query.lower()
    ) | df["full_name"].apply(lambda x: x.lower() == search_query.lower())
    exact_matches = df[exact_matches_mask]

    # Filter the DataFrame for rows containing all words
    contains_all_words_mask = df["preferred_name"].apply(
        lambda x: string_contains_all_words(x, query_words)
    ) | df["full_name"].apply(lambda x: string_contains_all_words(x, query_words))
    contains_all_words = df[contains_all_words_mask]

    # Concatenate the results to get the final output
    result = pd.concat([exact_matches, contains_all_words])

    return result

In [15]:
def get_player_data(df, player_name):
    # Filter the dataframe based on the name column
    filtered_df = search_dataframe(df, player_name)
    filtered_df = filtered_df.drop_duplicates()

    dfs_by_number = {}

    for id, group_df in filtered_df.groupby("ID"):
        dfs_by_number[id] = group_df

    dfs_by_search = []

    for id, separate_df in dfs_by_number.items():
        separate_df.sort_values(by=["opponent_team"], inplace=True, ascending=False)

        # Count the number of rows for each opponent_team
        count_df = separate_df["opponent_team"].value_counts().reset_index()
        count_df.columns = ["opponent_team", "matches_number"]

        sum_df = separate_df.groupby("opponent_team").agg(
            {"goals_scored": "sum", "assists": "sum", "threat": "sum"}
        )

        mean_df = separate_df.groupby("opponent_team").agg(
            {
                "influence": lambda x: round(x.mean(), 2),
                "creativity": lambda x: round(x.mean(), 2),
                "total_points": lambda x: round(x.mean(), 2),
                "weighted_points": lambda x: round(x.mean(), 2),
            }
        )

        # Merge the count_df, sum_df, and mean_df dataframes on 'opponent_team'
        new_df = pd.merge(count_df, sum_df, on="opponent_team")
        new_df = pd.merge(new_df, mean_df, on="opponent_team")

        new_df["full_name"] = separate_df["full_name"].iloc[0]
        new_df["preferred_name"] = separate_df["preferred_name"].iloc[0]
        new_df["ID"] = separate_df["ID"].iloc[0]

        # Rename the columns based on the aggregation method used
        new_df = new_df.rename(
            columns=lambda x: f"total_{x}"
            if x in sum_df.columns
            else (f"average_{x}" if x in mean_df.columns else x)
        )

        new_df.sort_values(by=["average_total_points"], inplace=True, ascending=False)

        new_df.reset_index(drop=True, inplace=True)
        new_df.index = new_df.index + 1

        dfs_by_search.append(new_df)

    final_df = pd.concat(dfs_by_search, ignore_index=False)
    desired_columns_order = [
        "full_name",
        "preferred_name",
        "ID",
        "opponent_team",
        "matches_number",
        "total_goals_scored",
        "total_assists",
        "total_threat",
        "average_influence",
        "average_creativity",
        "average_total_points",
        "average_weighted_points",
    ]
    final_df = final_df.reindex(columns=desired_columns_order)
    
    return final_df

In [16]:
def get_player_performance(team, player_name):
    # Get the list of fixtures for each team
    fixtures_all = fixtures_df[(fixtures_df['team_a'] == team) | (fixtures_df['team_h'] == team)]
    fixtures_all = fixtures_all.reset_index(drop=True)

    # Get only the fixtures ahead picked in numFix
    fixList = fixtures_all[(~fixtures_all['finished']).idxmax() : (~fixtures_all['finished']).idxmax() + numFix]

    # Get the unique team names from both columns
    opponent_teams = fixList['team_a'].append(fixList['team_h']).unique()
    opponent_teams = opponent_teams[opponent_teams != team]

    # use function to get the historical data of players' performances
    player_history_all = get_player_data(df, player_name)

    # filter by only the opponent_team in the ahead fixtures
    player_history = player_history_all[player_history_all['opponent_team'].isin(opponent_teams)]
    return player_history

# Get Fixtures

In [17]:
## Get fixtures - Request json data 
r2 = requests.get(url_fix)
json2 = r2.json()

In [18]:
# transform into dataframe
fixtures_df = pd.DataFrame(json2)

In [19]:
fixtures_df['team_a'] = fixtures_df.team_a.map(teams_df.set_index('id').name)
fixtures_df['team_h'] = fixtures_df.team_h.map(teams_df.set_index('id').name)

In [20]:
fixtures_df.head()

,code,event,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id
0,2367552,NaN,False,False,15,None,0,False,None,Burnley,None,Luton,None,[],2,2,93335
1,2367713,NaN,False,False,176,None,0,False,None,Brentford,None,Man City,None,[],3,5,93496
2,2367538,1.0,False,False,1,2023-08-11T19:00:00Z,0,False,False,Man City,None,Burnley,None,[],5,2,93321
3,2367540,1.0,False,False,2,2023-08-12T11:30:00Z,0,False,False,Nott'm Forest,None,Arsenal,None,[],2,4,93322
4,2367539,1.0,False,False,3,2023-08-12T14:00:00Z,0,False,False,West Ham,None,Bournemouth,None,[],2,2,93323


# Get players average historical performance against teams in the choosen number of fixture ahead

In [21]:
# Get all the team names
team_names = teams_df.name

In [22]:
# Num Fixtures to look ahead
numFix = 8

In [23]:
# Define column names -- Create empty dataframe to append
columns = ['total_goals_scored', 'total_assists', 'total_threat', 'average_influence', 'average_creativity', 'average_total_points','average_weighted_points']
final_df = pd.DataFrame(columns=columns)
final_df

,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points


In [24]:
# loop thru every team in the league
for team in team_names:

    # Get the list of fixtures for each team
    fixtures_all = fixtures_df[(fixtures_df['team_a'] == team) | (fixtures_df['team_h'] == team)]
    fixtures_all = fixtures_all.reset_index(drop=True)

    # Get only the fixtures ahead picked in numFix
    fixList = fixtures_all[(~fixtures_all['finished']).idxmax() : (~fixtures_all['finished']).idxmax() + numFix]

    # Get the unique team names from both columns
    opponent_teams = fixList['team_a'].append(fixList['team_h']).unique()
    opponent_teams = opponent_teams[opponent_teams != team]

    # Get the team rosters
    team_rosters = slim_elements_df['full_name'][slim_elements_df['team'] == team]
    team_rosters = team_rosters.reset_index(drop=True)

    # run thru every players in the team rosters
    for player_name in team_rosters:
        try:
            # use function to get the historical data of players' performances
            player_history_all = get_player_data(df, player_name)

            # filter by only the opponent_team in the ahead fixtures
            player_history = player_history_all[player_history_all['opponent_team'].isin(opponent_teams)]

            # get the mean data
            mean_player_history = player_history.groupby("full_name").agg(
                {
                    "total_goals_scored": lambda x: round(x.mean(), 2),
                    "total_assists": lambda x: round(x.mean(), 2),
                    "total_threat": lambda x: round(x.mean(), 2),
                    "average_influence": lambda x: round(x.mean(), 2),
                    "average_creativity": lambda x: round(x.mean(), 2),
                    "average_total_points": lambda x: round(x.mean(), 2),
                    "average_weighted_points": lambda x: round(x.mean(), 2),
                    "matches_number": lambda x: (x.sum()),
                })     

            # append data
            final_df = pd.concat([final_df, mean_player_history])

        except Exception as e:
            # Handle the error or just skip it
            continue  # Skip to the next iteration of the loop

In [25]:
# Search specific player for more details
team = 'Sheffield Utd'
player_name = 'Daniel David Jebbison'

# run the function to get the player's performances
get_player_performance(team, player_name)

,full_name,preferred_name,ID,opponent_team,matches_number,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points
1,Daniel David Jebbison,Daniel Jebbison,403,Everton,1,1,0,60,44.2,10.8,7.0,4.9
2,Daniel David Jebbison,Daniel Jebbison,403,Newcastle,1,0,0,0,7.0,2.6,2.0,1.4
4,Daniel David Jebbison,Daniel Jebbison,403,Crystal Palace,1,0,0,4,0.0,0.0,1.0,0.7


# Prepare data for Optimizing dataset

In [26]:
# Create the optimized_data
optimized_data = final_df

# Reset index without dropping the reset index
optimized_data = optimized_data.reset_index(drop=False)

# Change column name 
optimized_data.rename(columns={'index': 'full_name'}, inplace=True)

In [27]:
optimized_data.head()

,full_name,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points,matches_number
0,Folarin Jerry Balogun,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.0
1,Cedric Ricardo Alves Soares,0.25,0.62,26.12,6.90,5.91,1.39,0.66,86.0
2,Mohamed Naser Elsayed Elneny,0.00,0.00,8.88,0.87,0.85,0.25,0.11,67.0
3,Mohammed Naser Elsayed Elneny,0.00,0.33,5.33,6.00,2.80,1.38,1.05,22.0
4,Fabio Daniel Ferreira Vieira,0.00,0.00,11.75,1.93,5.14,0.69,0.69,16.0


In [52]:
## Map the player team, position, values to the dataset
# Merge dataframes based on fuzzy string matching of 'player name'
# Get the package
from fuzzywuzzy import fuzz, process

def fuzzy_merge(row, choices, scorer, threshold):
    match, score = process.extractOne(row['full_name'], choices, scorer=scorer)
    if score >= threshold:
        return match
    else:
        return None

threshold = 50  # Adjust the threshold as needed

choices = slim_elements_df['full_name'].tolist()

optimized_data['matching_player'] = optimized_data.apply(fuzzy_merge, args=(choices, fuzz.ratio, threshold), axis=1)

combined_df = pd.merge(optimized_data, slim_elements_df, left_on='matching_player', right_on='full_name', how='left')

combined_df.drop(columns=['matching_player'], inplace=True)

# divide the col now_cost by 10
combined_df['now_cost'] = combined_df['now_cost'] / 10

In [53]:
# delete some unnecessary columns/rows
# delete a column by specifying the column name and axis=1
column_to_delete = ['first_name', 'second_name', 'element_type', 'total_points', 'full_name_y']
combined_df = combined_df.drop(column_to_delete, axis=1)

# delete nan rows
combined_df = combined_df.dropna()

# add in matches played weighted average point columns
combined_df['weighted_avg_point'] = round((combined_df['matches_number']/combined_df['matches_number'].sum()) * 
                                          (combined_df['average_total_points']*combined_df['matches_number']),4)

# add in cost paid for 1 point
combined_df['point_per_mil'] = round((combined_df['weighted_avg_point'] / combined_df['now_cost']),4)

In [222]:
# Create the list of players I want to put in the team 
player_self_picked_budget = 9.5 + 14 + 5 + 4.5 + 5 + 5.5
self_picked_players = ['Haaland','Shaw','Estupinan','Gabriel','Steele','Martinez','Cash']
# player_self_picked_num
Def_num = 4
Fwd_num = 1
Mid_num = 0
Gk_num = 2

# pos_budget -- either set a number or leave it at 100
Def_budget = 100
Fwd_budget = 100
Mid_budget = 100
Gk_budget = 10

In [223]:
# Enter a list of player who is not counting anymore
blacklist_player = ['Ivan Benjamin Elijah Toney','Daniel Johnson Burn','Romelu Lukaku Bolingoli','Hugo Hadrien Dominique Lloris','David Raya Martin','Joao Pedro Cavaco Cancelo','Lucas Tolentino Coelho de Lima','Pervis Josue Estupinan Tenorio','Julian Alvarez','Erling Braut Haaland']

In [224]:
# Filter out only players with specific amount of matches played
combined_df_final = combined_df[combined_df['matches_number'] >= 10]

# Remove players in the blacklists
combined_df_final = combined_df_final[~combined_df_final['full_name_x'].isin(blacklist_player)]

In [225]:
combined_df_final.head()

,full_name_x,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points,matches_number,team,selected_by_percent,now_cost,transfers_in,value_season,position,weighted_avg_point,point_per_mil
0,Folarin Jerry Balogun,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.0,Arsenal,1.3,4.5,0.0,0.0,Forward,0.0000,0.0000
1,Cedric Ricardo Alves Soares,0.25,0.62,26.12,6.90,5.91,1.39,0.66,86.0,Arsenal,0.4,4.0,0.0,2.5,Defender,0.5126,0.1281
2,Mohamed Naser Elsayed Elneny,0.00,0.00,8.88,0.87,0.85,0.25,0.11,67.0,Arsenal,0.2,4.5,0.0,1.3,Midfielder,0.0560,0.0124
3,Mohammed Naser Elsayed Elneny,0.00,0.33,5.33,6.00,2.80,1.38,1.05,22.0,Arsenal,0.2,4.5,0.0,1.3,Midfielder,0.0333,0.0074
4,Fabio Daniel Ferreira Vieira,0.00,0.00,11.75,1.93,5.14,0.69,0.69,16.0,Arsenal,0.1,5.5,0.0,7.3,Midfielder,0.0088,0.0016


# Optimizer Setup

In [226]:
# Factor selected for optimizing
optimized_object = 'average_weighted_points'

In [227]:
# import necessary packages
from pulp import LpVariable, LpProblem, lpSum, LpMaximize

In [205]:
# Initialize variables
# Helper variables
POS = combined_df_final.position.unique()
CLUBS = combined_df_final.team.unique()
BUDGET = 100 - player_self_picked_budget
pos_available = {
    'Defender': 5 - Def_num,
    'Forward': 3 - Fwd_num,
    'Midfielder': 5 - Mid_num,
    'Goalkeeper': 2 - Gk_num,
}
pos_budget = {
    'Defender': Def_budget,
    'Forward': Fwd_budget,
    'Midfielder': Mid_budget,
    'Goalkeeper': Gk_budget,
}

# Initialize Variables
names = [combined_df_final.full_name_x[i] for i in combined_df_final.index]
teams = [combined_df_final.team[i] for i in combined_df_final.index]
positions = [combined_df_final.position[i] for i in combined_df_final.index]
prices = [combined_df_final.now_cost[i] for i in combined_df_final.index]
points = [combined_df_final[optimized_object][i] for i in combined_df_final.index]
players = [LpVariable("player_" + str(i), cat="Binary") for i in combined_df_final.index]

In [206]:
# Initialize the problem
prob = LpProblem("FPL Player Choices", LpMaximize)

In [207]:
# Define the objective
prob += lpSum(players[i] * points[i] for i in range(len(combined_df_final))) # Objective

In [208]:
# Build the constraints
prob += lpSum(players[i] * combined_df_final.now_cost[combined_df_final.index[i]] for i in range(len(combined_df_final))) <= BUDGET # Budget Limit

for pos in POS:
    prob += lpSum(players[i] for i in range(len(combined_df_final)) if positions[i] == pos) <= pos_available[pos] # Position Limit
    prob += lpSum(players[i] * combined_df_final.now_cost[combined_df_final.index[i]] for i in range(len(combined_df_final)) if positions[i] == pos) <= pos_budget[pos] # Position Price Limit
for club in CLUBS:
    prob += lpSum(players[i] for i in range(len(combined_df_final)) if teams[i] == club) <= 3 # Club Limit

In [209]:
# Solve the problem
prob.solve()

1

# Get the results

In [244]:
# Retrieve the list of players through optimization
points = []
prices = []
names = []
clubs = []
for v in prob.variables():
    if v.varValue != 0:
        name = combined_df_final.full_name_x[int(v.name.split("_")[1])]
        club = combined_df_final.team[int(v.name.split("_")[1])]
        position = combined_df_final.position[int(v.name.split("_")[1])]
        point = combined_df_final[optimized_object][int(v.name.split("_")[1])]
        points.append(point)
        price = combined_df_final.now_cost[int(v.name.split("_")[1])]
        prices.append(price)
        names.append(name)
        clubs.append(club)
        print(name, position, club, point, price, sep=" | ")

Mathias Jensen | Midfielder | Brentford | 3.09 | 5.5
Kaoru Mitoma | Midfielder | Brighton | 4.14 | 6.5
Martin Odegaard | Midfielder | Arsenal | 4.54 | 8.5
Raheem Shaquille Sterling | Midfielder | Chelsea | 3.63 | 7.0
Cody Mathes Gakpo | Forward | Liverpool | 4.19 | 7.5
Ruben dos Santos Gato Alves Dias | Defender | Man City | 3.36 | 5.5
Antony Matheus dos Santos | Midfielder | Man Utd | 3.71 | 7.0
Oliver George Arthur Watkins | Forward | Aston Villa | 3.73 | 8.0


In [211]:
print(f"The amount of budget spend is {sum(prices) + player_self_picked_budget}")
print(f"The amount of total estimated points is {round(sum(points),2)}")

The amount of budget spend is 99.0
The amount of total estimated points is 30.39


In [67]:
# Get the list of best Goalkeeper 
combined_df_final[combined_df_final['position'] == 'Goalkeeper'].sort_values(by=optimized_object, ascending=False)[0:10]

,full_name_x,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points,matches_number,team,selected_by_percent,now_cost,transfers_in,value_season,position,weighted_avg_point,point_per_mil
52,Damian Emiliano Martinez Romero,0.0,0.0,0.0,24.04,0.93,4.14,4.14,14.0,Aston Villa,7.9,5.0,0.0,27.0,Goalkeeper,0.0405,0.0081
114,David Raya Martin,0.0,0.0,0.5,23.72,0.00,3.16,3.01,26.0,Brentford,8.1,5.0,0.0,33.2,Goalkeeper,0.1065,0.0213
321,Ederson Santana de Moraes,0.0,0.0,0.0,11.44,0.31,4.13,2.96,60.0,Man City,14.1,5.5,0.0,22.0,Goalkeeper,0.7414,0.1348
280,Alisson Ramses Becker,0.0,0.0,0.0,18.81,0.00,3.94,2.78,74.0,Liverpool,9.1,5.5,0.0,29.5,Goalkeeper,1.0759,0.1956
532,Jose Pedro Malheiro de Sa,0.0,0.0,0.0,23.14,0.00,2.96,2.72,28.0,Wolves,1.3,5.0,0.0,29.6,Goalkeeper,0.1157,0.0231
173,Robert Lynch Sanchez,0.0,0.0,0.0,13.04,0.36,3.00,2.61,30.0,Chelsea,4.3,4.5,0.0,18.2,Goalkeeper,0.1346,0.0299
18,Aaron Christopher Ramsdale,0.0,0.0,0.0,17.50,0.17,2.84,2.47,57.0,Arsenal,20.4,5.0,0.0,28.6,Goalkeeper,0.4601,0.0920
212,Vicente Guaita Panadero,0.0,0.0,0.0,14.91,0.00,3.12,2.36,54.0,Crystal Palace,1.2,4.5,0.0,22.0,Goalkeeper,0.4537,0.1008
246,Jordan Lee Pickford,0.0,0.0,0.0,21.20,0.92,3.11,2.16,56.0,Everton,12.7,4.5,0.0,27.6,Goalkeeper,0.4863,0.1081
79,Norberto Murara Neto,0.0,0.0,0.0,17.42,0.00,2.00,2.00,16.0,Bournemouth,0.8,4.5,0.0,20.7,Goalkeeper,0.0255,0.0057


In [178]:
# Get the list of best Defender 
combined_df_final[(combined_df_final['position'] == 'Defender') & (combined_df_final['now_cost'] <= 5)].sort_values(by=optimized_object, ascending=False)[0:10]

,full_name_x,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points,matches_number,team,selected_by_percent,now_cost,transfers_in,value_season,position,weighted_avg_point,point_per_mil
131,Pervis Josue Estupinan Tenorio,0.00,0.14,12.14,14.27,21.77,3.64,3.64,12.0,Brighton,51.5,5.0,0.0,25.6,Defender,0.0261,0.0052
202,Thiago Emiliano da Silva,0.29,0.14,24.71,22.84,3.53,3.82,3.25,30.0,Chelsea,2.0,5.0,0.0,16.0,Defender,0.1714,0.0343
5,Gabriel dos Santos Magalhaes,0.50,0.00,41.75,16.86,4.17,3.33,3.04,38.0,Arsenal,26.3,5.0,0.0,29.2,Defender,0.2398,0.0480
151,Joel Ivo Veltman,0.29,0.14,15.29,12.57,6.08,2.98,2.71,38.0,Brighton,1.3,4.5,0.0,21.1,Defender,0.2146,0.0477
365,Raphael Xavier Varane,0.12,0.12,13.88,13.98,2.03,2.78,2.56,28.0,Man Utd,1.7,5.0,0.0,15.4,Defender,0.1087,0.0217
494,Vladimir Coufal,0.00,0.71,27.71,13.29,11.61,2.87,2.43,33.0,West Ham,0.3,4.5,0.0,12.9,Defender,0.1559,0.0346
248,James Alan Tarkowski,0.43,0.14,80.57,25.63,3.26,3.35,2.39,56.0,Everton,2.7,4.5,0.0,23.6,Defender,0.5239,0.1164
104,Rico Antonio Henry,0.25,0.12,31.00,13.46,10.90,2.59,2.39,26.0,Brentford,4.8,4.5,0.0,24.9,Defender,0.0873,0.0194
113,Ethan Rupert Pinnock,0.25,0.00,23.75,19.61,2.00,2.50,2.38,26.0,Brentford,1.9,4.5,0.0,24.9,Defender,0.0843,0.0187
39,Matthew Stuart Cash,0.25,0.25,45.25,13.58,9.27,2.83,2.35,46.0,Aston Villa,5.6,4.5,0.0,12.7,Defender,0.2986,0.0664


In [256]:
# Get the list of best Midfielder 
combined_df_final[(combined_df_final['position'] == 'Midfielder') & (combined_df_final['now_cost'] <= 6.5)].sort_values(by=optimized_object, ascending=False)[0:15]

,full_name_x,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points,matches_number,team,selected_by_percent,now_cost,transfers_in,value_season,position,weighted_avg_point,point_per_mil
143,Kaoru Mitoma,0.43,0.57,44.57,14.47,9.73,4.14,4.14,14.0,Brighton,37.9,6.5,0.0,21.2,Midfielder,0.0405,0.0062
107,Mathias Jensen,0.25,0.38,17.50,15.11,18.84,3.16,3.09,26.0,Brentford,0.9,5.5,0.0,24.0,Midfielder,0.1065,0.0194
277,Willian Borges da Silva,0.17,0.50,32.83,11.88,18.47,3.00,3.00,10.0,Fulham,0.6,5.5,0.0,21.1,Midfielder,0.0150,0.0027
314,Bernardo Mota Veiga de Carvalho e Silva,0.75,1.88,132.00,15.51,18.77,3.60,2.72,60.0,Man City,4.1,6.5,0.0,16.3,Midfielder,0.6463,0.0994
211,Eberechi Oluchi Eze,0.62,0.50,61.62,14.15,14.65,3.29,2.69,36.0,Crystal Palace,15.6,6.5,0.0,24.5,Midfielder,0.2126,0.0327
109,Bryan Tetsadong Marceau Mbeumo,0.38,0.38,67.00,11.94,19.85,2.75,2.65,26.0,Brentford,15.4,6.5,0.0,23.1,Midfielder,0.0927,0.0143
507,Tomas Soucek,0.71,0.29,109.00,18.80,8.68,3.22,2.64,39.0,West Ham,1.1,5.0,0.0,20.0,Midfielder,0.2442,0.0488
405,Harvey Lewis Barnes,1.88,1.75,195.88,15.57,10.65,3.55,2.61,61.0,Newcastle,2.3,6.5,0.0,21.2,Midfielder,0.6587,0.1013
242,Dwight James Matthew McNeil,0.71,1.29,74.29,17.33,21.85,3.39,2.60,48.0,Everton,0.5,5.5,0.0,22.9,Midfielder,0.3895,0.0708
53,John McGinn,0.75,1.38,82.38,17.13,16.13,3.16,2.46,62.0,Aston Villa,0.5,5.5,0.0,16.4,Midfielder,0.6057,0.1101


In [50]:
# Get the list of best Forward 
combined_df_final[combined_df_final['position'] == 'Forward'].sort_values(by=optimized_object, ascending=False)[0:10]

,full_name_x,total_goals_scored,total_assists,total_threat,average_influence,average_creativity,average_total_points,average_weighted_points,matches_number,team,selected_by_percent,now_cost,transfers_in,value_season,position,weighted_avg_point,point_per_mil
324,Erling Braut Haaland,2.17,0.17,89.50,38.63,9.72,7.25,7.25,12.0,Man City,86.6,14.0,0.0,19.4,Forward,0.0521,0.5179
286,Cody Mathes Gakpo,0.25,0.25,22.25,15.24,12.01,4.19,4.19,10.0,Liverpool,6.0,7.5,0.0,12.0,Forward,0.0209,0.5587
62,Oliver George Arthur Watkins,2.38,1.25,210.62,22.16,13.57,4.46,3.73,46.0,Aston Villa,22.5,8.0,0.0,21.9,Forward,0.4706,0.5575
313,Julian Alvarez,0.67,0.17,43.50,16.12,8.06,3.50,3.50,12.0,Man City,3.5,6.5,0.0,15.8,Forward,0.0251,0.5385
409,Taiwo Micheal Awoniyi,0.67,0.00,22.00,13.98,3.24,3.08,3.08,12.0,Nott'm Forest,1.1,6.5,0.0,15.4,Forward,0.0221,0.4738
120,Yoane Wissa,0.50,0.50,35.38,8.63,4.14,3.16,3.04,26.0,Brentford,4.6,6.0,0.0,18.5,Forward,0.1065,0.5267
466,Harry Edward Kane,5.14,1.43,444.57,24.61,15.56,4.68,2.89,72.0,Spurs,13.5,12.5,0.0,21.0,Forward,1.2098,0.3744
292,Luis Fernando Diaz Marulanda,0.62,0.25,57.00,12.88,7.81,2.96,2.75,22.0,Arsenal,9.2,8.0,0.0,15.6,Forward,0.0714,0.3700
385,Alexander Isak,0.50,0.00,27.33,12.25,4.19,2.33,2.33,10.0,Newcastle,14.9,7.5,0.0,13.3,Forward,0.0116,0.3107
402,Callum Eddie Graham Wilson,3.25,1.50,216.62,12.57,9.68,3.38,2.32,82.0,Newcastle,9.9,8.0,0.0,19.6,Forward,1.1333,0.4225


In [254]:
# Search specific player for more details
# Define column names -- Create empty dataframe to append
columns_temp = ['full_name', 'opponent_team', 'average_weighted_points']
final_df_temp = pd.DataFrame(columns=columns_temp)

for i in range(0,len(clubs)-1):
    team = clubs[i]
    player_name = names[i]

    # run the function to get the player's performances
    player_performance = get_player_performance(team, player_name)
    final_df_temp = pd.concat([final_df_temp, player_performance])
    

In [255]:
print(final_df_temp[['full_name', 'opponent_team', 'average_weighted_points']])

                           full_name   opponent_team  average_weighted_points
1                     Mathias Jensen          Fulham                     7.00
2                     Mathias Jensen     Bournemouth                     6.50
6                     Mathias Jensen   Nott'm Forest                     3.50
8                     Mathias Jensen           Spurs                     2.34
13                    Mathias Jensen  Crystal Palace                     1.63
17                    Mathias Jensen         Everton                     1.42
19                    Mathias Jensen       Newcastle                     1.21
20                    Mathias Jensen        Man City                     1.09
1                       Kaoru Mitoma     Bournemouth                     9.00
2                       Kaoru Mitoma        West Ham                     7.00
5                       Kaoru Mitoma       Liverpool                     5.00
7                       Kaoru Mitoma          Wolves            